In [2]:
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from glob import glob
import os
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd
import cv2
import math

In [3]:
fc.remove_thumbs("//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset/chips_positive")

In [14]:
def collected_quads_to_tile_name_tile_url(quads):
    """
    Read in a excel sheet which includes the quadrangle 
    """
    
    tile_name = []
    tile_url = []
    file_name_index = {'m': 0, 'qqname': 1, 'direction': 2,'YY': 3, 'resolution': 4,'capture_date': 5,'version_date': 5}
    blob_root = 'https://naipeuwest.blob.core.windows.net/naip'
    two_digit_state_resolution = ["al","ak","az","ar","ca", "co","ct","de","fl","ga",
                                  "hi","id","il","in","ia", "ks","ky","la","me","md",
                                  "ma","mi","mn","ms","mo", "mt","ne","nv","nh","nj",
                                  "nm","ny","nc","nd","oh", "ok","or","pa","ri","sc",
                                  "sd","tn","tx","ut","vt", "va", "wa","wv","wi","wy"]

    for i in range(len(quads)):
        file_name = quads.iloc[i,0].split('_') #filename
        state = quads.iloc[i,1].lower() #state
        year = quads.iloc[i,2] # YYYY
        if state in two_digit_state_resolution:
            resolution = file_name[file_name_index["resolution"]][1:3]+"cm"
        else:
            resolution = file_name[file_name_index["resolution"]]+"cm"
        quadrangle = file_name[file_name_index["qqname"]][0:5] #qqname
        
        tile_name.append(quads.iloc[i,0] +'.tif')
        tile_url.append(blob_root + '/v002/' + state + '/' + str(year)+ '/' + state + '_' + resolution \
                    + '_' + str(year) + '/' + str(quadrangle) + '/' + tile_name[i])
        # Tiles are stored at: [blob root]/v002/[state]/[year]/[state]_[resolution]_[year]/[quadrangle]/filename
    return (tile_name, tile_url)

In [11]:
quads = pd.read_csv("image_download_azure/HSC.csv")

In [25]:
tile_name, tile_url = collected_quads_to_tile_name_tile_url(quads)

In [30]:
tile_name_tile_url = np.array((tile_name, tile_url)).T

In [17]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

#create folder to hold tiles in completed dataset
#load tiles and tile xmls
tiles_dir = os.path.join(parent_directory,"complete_dataset","tiles")
tiles = os.listdir(tiles_dir)

In [46]:
file_names = []
for tile in tile_name_tile_url:
    quads = tile[0].rsplit("_",1)[0]
    tile_download = [string for string in tiles if quads in string]  
    if len(tile_download) < 1:
        file_names.append(ap.download_url(tile[1], tiles_dir,
                                          destination_filename = tile[0],       
                                          progress_updater=ap.DownloadProgressBar()))

HTTPError: HTTP Error 404: The specified blob does not exist.

In [43]:
Z:\temp\set1

array([['m_2909515_se_15_060_20201130.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909515_se_15_060_20201130.tif'],
       ['m_2909515_sw_15_060_20201130.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909515_sw_15_060_20201130.tif'],
       ['m_2909516_sw_15_060_20201118.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909516_sw_15_060_20201118.tif'],
       ['m_2909522_ne_15_060_20201130.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909522_ne_15_060_20201130.tif'],
       ['m_2909523_nw_15_060_20201130.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909523_nw_15_060_20201130.tif'],
       ['m_2909523_ne_15_060_20201130.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2020/tx_60cm_2020/29095/m_2909523_ne_15_060_20201130.tif']

In [55]:
chips_path = os.path.join(parent_directory, "temp/set1/chips")
tiles_path = os.path.join(parent_directory, "temp/set1/tiles")

for tile in os.listdir(tiles_path):
    item_dim = int(512)
    tile_name = os.path.splitext(tile)[0]

    tile = cv2.imread(os.path.join(tiles_path, tile)) 
    tile_height,  tile_width,  tile_channels = tile.shape #the size of the tile 
    row_index = math.ceil(tile_height/512) 
    col_index = math.ceil(tile_width/512)
    #print(row_index, col_index)

    for y in range(0, row_index): #rows
        for x in range(0, col_index): #cols
            chip_img = fc.tile_to_chip_array(tile, x, y, item_dim)
            #specify the chip names
            chip_name_correct_chip_name = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" + '.jpg' # The index is a six-digit number like '000023'.
            if not os.path.exists(os.path.join(chips_path, chip_name_correct_chip_name)):
                cv2.imwrite(os.path.join(chips_path, chip_name_correct_chip_name), chip_img) #save images  

In [ ]:
   
    ### download the tiles if they are not in the tiles folder 
    #check if the tile name is contained in the string of complete arrays
        